In [1]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import mapping
import folium
import gmaps
import os
import requests
import shutil

In [2]:
with open('../resources/key.txt') as f:
    token = f.read()
    
gmaps.configure(api_key = token)

In [50]:
'''
Set local I/O
'''

# Set folder and filename for rail GIS file
rail_folder = '../data/railways/Amtrak_Routes/'
rail_file = rail_folder+'Amtrak_Routes.shp'

# Set folder and filename for rail CSV file
output_folder = '../data/output_images/Amtrak/'
output_csv = output_folder+'AMT.csv'

# Set folder for new images
image_folder = output_folder+'set_3/'

### Import Routes

In [51]:
routes = gpd.read_file(rail_file)
routes = routes.to_crs({'init' :'epsg:4269'})
routes.head()

,OBJECTID,NAME,Shape_Leng,ShapeSTLen,geometry
0,1,Acela,7.410105e+05,9.779207e+05,(LINESTRING (-77.01421252564722 38.88359551476...
1,2,Adirondack,6.156754e+05,8.437497e+05,(LINESTRING (-73.74197100078742 42.64026759195...
2,3,Auto Train,1.474024e+06,1.776112e+06,(LINESTRING (-81.3176960592689 28.758923120780...
3,4,Blue Water,5.114384e+05,6.939223e+05,(LINESTRING (-87.63609509602139 41.81771709246...
4,5,California Zephyr,4.313889e+06,5.672842e+06,(LINESTRING (-108.5558964471134 39.06262618765...


In [42]:
routes.shape

(6, 6)

In [53]:
'''
Get all points
'''

points = []
rows = routes.index.tolist()

geom_types = routes['geometry'].tolist()
for i in rows:
    try:
        route = routes.loc[i]
        if route.geometry.geom_type == 'LineString':
            g = route.geometry
            sec_points = mapping(g)["coordinates"]
            for j in sec_points:
                tmp = (j[0],j[1])
                points.append(tmp)
        elif route.geometry.geom_type == 'MultiLineString':
            g = route.geometry
            tmp = mapping(g)["coordinates"]
            for sec_points in tmp:
                for j in sec_points:
                    tmp = (j[0],j[1])
                    points.append(tmp)
    except Exception as e:
        print("Skipped route number",i,'because',e)        
        
    
print(len(points),'points')   

608944 points


In [54]:
'''
Get points for 1 or more routes
'''

rows = [1]

points = []

geom_types = routes['geometry'].tolist()
for i in rows:
    try:
        route = routes.loc[i]
        if route.geometry.geom_type == 'LineString':
            g = route.geometry
            sec_points = mapping(g)["coordinates"]
            for j in sec_points:
                tmp = (j[0],j[1])
                points.append(tmp)
        elif route.geometry.geom_type == 'MultiLineString':
            g = route.geometry
            tmp = mapping(g)["coordinates"]
            for sec_points in tmp:
                for j in sec_points:
                    tmp = (j[0],j[1])
                    points.append(tmp)
    except Exception as e:
        print("Skipped route number",i,'because',e)        
        
    
print(len(points),'points')

9792 points


##### Get nth number

In [55]:
'''
Open csv with current points
'''

curr_df = pd.read_csv(output_csv)
curr_df.head()

,Name,Longitude,Latitude,Catenary
0,-71.0986135186152_42.32525050375276,-71.098614,42.325251,1
1,-71.1537173899704_42.160718367371224,-71.153717,42.160718,1
2,-71.28022732996062_41.94747480732387,-71.280227,41.947475,1
3,-71.41494832260199_41.84486643575766,-71.414948,41.844866,1
4,-71.42193732420859_41.78370544868109,-71.421937,41.783705,1


In [58]:
'''
Get subselection of points excuding current points
'''
rows = curr_df.index.tolist()
curr_points = []
for row in rows:
    tmp = curr_df.iloc[row].Name
    space = tmp.find('_') 
    longitude = tmp[:space]
    latitude = tmp[space+1:]
    curr_points.append((longitude,latitude))

route_points = []
for point in range(0,len(points),95):
    tmp = points[point]
    if tmp not in curr_points:
        route_points.append(tmp)

print(len(route_points))

104


In [59]:
'''
Create dataframe
'''

df = pd.DataFrame(route_points, columns=['Longitude','Latitude'])

tmp = list(df.Latitude.tolist())
names = []
for i in range(0,len(tmp)):
    name = str(df.iloc[i].Longitude)+'_'+str(df.iloc[i].Latitude)
    names.append(name)

df['Name'] = names

columns = ['Name','Longitude','Latitude']
df= df[columns]

df.head()

,Name,Longitude,Latitude
0,-73.74197100078742_42.6402675919547,-73.741971,42.640268
1,-73.75778418127769_42.529043187624474,-73.757784,42.529043
2,-73.78037619083574_42.400564289420146,-73.780376,42.400564
3,-73.77634552202112_42.346381421677485,-73.776346,42.346381
4,-73.78150610437652_42.27707356676576,-73.781506,42.277074


In [60]:
'''
Plot markers
'''

rows = df.index.tolist()

marker_points = []
for row in rows:
    marker_points.append((df.iloc[row].Latitude,df.iloc[row].Longitude))
    
# Set map centerpoint
coords = marker_points[0]
       
# Define map
m = folium.Map(
    location = coords,
    zoom_start = 13
)
  
# Add points    
for mp in marker_points:
    folium.Marker(mp).add_to(m)
m

In [24]:
'''
Get satellite preview for image
'''

row = 1

figure_layout = {
    'width': '100%',
    'height': '800px'
}
# fig = gmaps.figure(center = (df.iloc[row].Latitude,df.iloc[row].Longitude), zoom_level = 19,map_type='SATELLITE',layout=figure_layout)
fig = gmaps.figure(center = (list(routes[routes['sub_type']=='1'].iloc[5].geometry.coords)[0][1],
                            list(routes[routes['sub_type']=='1'].iloc[5].geometry.coords)[0][0]), zoom_level = 19,map_type='SATELLITE',layout=figure_layout)

fig

Figure(layout=FigureLayout(height='800px', width='100%'))

In [61]:
'''
Export points
'''

# Update
with open(output_csv, 'a') as f:
    df.to_csv(f, header=False,index=None)

# Write new
# df.to_csv(output_csv,header=True,index=None)

In [62]:
'''
Get all images 
'''

root = os.path.dirname(os.path.abspath('Collection.ipynb'))
img_folder = root[:-9]+image_folder[3:]
img_folder

url = 'https://maps.googleapis.com/maps/api/staticmap?'
rows = df.index.tolist()
for i in range(0,len(rows)):
    row = df.iloc[i]
    center = str(row.Latitude)+','+str(row.Longitude)
    payload = {
        'center': center, 
        'zoom': 20,
        'size': '640x640',
        'scale': 2,
        'format': 'png32',
        'maptype': 'satellite',
        'key': token
    }
    
    r = requests.get(url,params=payload,stream=True)

    name = df.iloc[i]['Name']
    filename = img_folder+name+'.png'
    if r.status_code == 200:
        with open(filename, 'wb') as f:
            r.raw.decode_content = True
            shutil.copyfileobj(r.raw, f) 
    else:
        print(r.status_code)